## DATA

In [1]:
# https://nirvana.yandex-team.ru/flow/73004777-a701-43c5-8707-6f1adfb8402b/c36b1394-d342-46c8-925c-595d66a359d4/graph/FlowchartEndpoint/f95c783d-bd0e-447f-82b6-56fb87fe0618
yt_path = "//home/verticals/.tmp/realty/price_estimator/price_estimator_pool/catboost_yt_table__LosFeau8R1KFOrnjhgNGQQ"

In [2]:
!yt --proxy hahn read $yt_path --format='<columns=[key;value]; enable_column_names_header=%true; missing_value_mode=print_sentinel; missing_value_sentinel="">schemaful_dsv' > ya_realty_test_pool2.tsv
 

In [3]:
# https://nirvana.yandex-team.ru/flow/73004777-a701-43c5-8707-6f1adfb8402b/c36b1394-d342-46c8-925c-595d66a359d4/graph/FlowchartEndpoint/a6dee0dd-189a-46fa-bc0c-7bcb46f1e779
yt_path = " //home/verticals/.tmp/realty/price_estimator/output1__qednqyThSWO1L7efJQUpDg"

In [4]:
!yt --proxy hahn read $yt_path --format='<columns=[offer_id;unified_address]; enable_column_names_header=%true; missing_value_mode=print_sentinel; missing_value_sentinel="">schemaful_dsv' > ya_realty_test_address2.tsv
 

## IMPORT

In [1]:
import pandas as pd
import numpy as np

In [2]:
import json

In [3]:
from catboost import CatBoostRegressor

In [4]:
import sys
import subprocess

In [8]:
sys.path.insert(0, "../../vertis-ml/realty/price_estimator/price-estimator-rest-api")
sys.path.insert(0, "../../vertis-ml/realty/price_estimator/price-estimator-rest-api/proto")

In [9]:
from proto.realty.offer import RealtySchema_pb2
from realty.prediction.price_prediction_pb2 import PricePredictionResponse, PricePredictionLandingRequest

## FUNC

In [23]:
def get_service_prediction(row, offer_id, unified_address):
    message = RealtySchema_pb2.OfferMessage(
        offer_id=offer_id,
        apartmentInfo=RealtySchema_pb2.ApartmentInfoMessage(
            apartment="apartment" if row["is_apartment"] == "True" else "",
            studio=row["studio"] == "True",
            floors=[int(row["floor"])],
            ceilingHeight=row["ceiling_height"],
            roomsOffered=int(row["rooms_offered"]),
            rooms=int(row["rooms"]),
            balconyInt=int(row["balcony"]),
            renovationInt=int(row["renovation"]),
            flatTypeInt=int(row["flat_type"])
        ),
        offerTypeInt=2,
        categoryTypeInt=2,
        buildingInfo=RealtySchema_pb2.BuildingInfoMessage(
            floorsTotal=int(row["floors_total"]),
            buildingId=int(row["building_id"]),
            buildYear=int(row["build_year"]),
            buildingSeriesId=int(row["building_series_id"]),
            expectDemolition=row["expect_demolition"] == "True",
            parkingTypeInt=int(row["parking"]),
            buildingTypeInt=int(row["building_type"]),
            flatsCount=int(row["flats_count"]),
            site_id=int(row["site_id"]),
    #         has_elevator=??
        ),
        location=RealtySchema_pb2.LocationMessage(
            localityName=row["locality_name"],
            subjectFederationId=int(row["subject_federation_id"]),
    #         regionGraphId=message.location.regionGraphId,
            geocoderAddress=unified_address,
            geocoderPoint=RealtySchema_pb2.GeoPointMessage(
                latitude=row["lat"],
                longitude=row["lon"]
            )
        ),
    #     lotInfo=RealtySchema_pb2.LotInfoMessage(
    #         lotArea=RealtySchema_pb2.AreaInfoMessage(
    #             unitInt=message.lotInfo.lotArea.unitInt,
    #             value=message.lotInfo.lotArea.value
    #         ),
    #         lotTypeInt=message.lotInfo.lotTypeInt
    #     ),
        houseInfo=RealtySchema_pb2.HouseInfoMessage(
    #         pmg=message.houseInfo.pmg,
    #         houseTypeInt=message.houseInfo.houseTypeInt,
            livingSpace=row["living_area"],
            kitchenSpace=row["kitchen_area"]
        ),
        area=RealtySchema_pb2.AreaInfoMessage(value=row["total_area"]),
        transaction=RealtySchema_pb2.TransactionMessage(
            area=RealtySchema_pb2.AreaInfoMessage(value=row["area"])
        )
    )

    result = message.SerializeToString()

    request_filepath='proto.request.example.binary'
    with open(request_filepath, 'w+b') as file:
        file.write(result)

    response_filepath='proto.response.rent.example.binary'

    curl_call_string = f"""
    curl -s --header "Content-Type: application/protobuf" --request POST \
        --data-binary @{request_filepath} \
        http://localhost:8895/api/v2/get_price_offer > {response_filepath}
    """

    subprocess.check_call(curl_call_string, shell=True)


    price_resp = PricePredictionResponse()
    with open(response_filepath, 'r+b') as read_file:
        binary_string = read_file.read()
    price_resp.ParseFromString(binary_string)
    return price_resp.predicted_price.value

## CODE

данные

In [10]:
df_raw = pd.read_csv("ya_realty_test_pool2.tsv", sep="\t")

In [11]:
df_address = pd.read_csv("ya_realty_test_address2.tsv", sep="\t")

In [12]:
unified_address_dict = df_address.set_index("offer_id")["unified_address"].to_dict()

моделька

In [13]:
model_path = "rent_apartment.cbm"

In [14]:
cat = CatBoostRegressor().load_model(model_path, format='cbm')

In [15]:
model_factors = cat.feature_names_
num_features = model_factors[:cat.get_cat_feature_indices()[0]]
cat_features = [model_factors[ind] for ind in cat.get_cat_feature_indices()]

форматирование

In [16]:
df_vals = df_raw["value"].str.split(r"\\t").apply(pd.Series)

In [17]:
df_vals.columns = ["target", "id"] + model_factors + ["offer_id"]

In [22]:
for col in num_features:
    df_vals[col] = df_vals[col].astype("float")

In [18]:
df_vals.head()

,target,id,area,balcony,build_year,ceiling_height,flats_count,floors_total,total_area,living_area,...,city_quadkey,region_quadkey,building_id,building_series_id,building_type,parking,site_id,renovation,studio,offer_id
0,10.9681982895,2857917760913822208,64.0999984741,2,2009,3.0,0,23,64.0999984741,0.0,...,120310101020331,12031010102,3353672915864373485,1564812,2,8,0,7,False,2857917760913822208
1,10.0732303271,5883724591748225536,36.0,0,1984,2.59999990463,357,12,36.0,0.0,...,120121213030311,12012121303,2264734232811879417,1564801,4,8,0,7,False,5883724591748225536
2,10.8473934514,5265555201340415488,89.0,1,2004,0.0,0,7,89.0,0.0,...,120121213002021,12012121300,7912817987187760803,1564796,4,8,0,10,False,5265555201340415488
3,11.2897819137,5249989827891051264,67.6999969482,0,1955,3.0,331,16,67.6999969482,0.0,...,120310101120232,12031010112,1511878177171436665,1564812,1,8,0,7,False,5249989827891051264
4,11.1633679888,3160350346613730304,67.0,1,1958,3.0,112,8,67.0,0.0,...,120310101120213,12031010112,7986709337320943660,1564812,1,8,0,7,False,3160350346613730304


### 1 check service

In [24]:
results = []
for i,row in df_vals.iterrows():
    offer_id = row["offer_id"]
    unified_address = unified_address_dict[int(offer_id)]
    prediction = get_service_prediction(row, offer_id, unified_address)
    results.append(prediction)

In [46]:
preds = np.array(results)

In [84]:
y_true = np.exp(df_vals["target"].values.astype("float"))

In [85]:
np.mean(np.abs(y_true - preds) / y_true)

0.171077255987295

### 2 check local model

In [77]:
preds = np.exp(cat.predict(df_vals[model_factors]))

In [78]:
y_true = np.exp(df_vals["target"].values.astype("float"))

In [79]:
np.mean(np.abs(y_true - preds) / y_true)

0.17322823479871527

in [nirvana graph](https://nirvana.yandex-team.ru/flow/73004777-a701-43c5-8707-6f1adfb8402b/c36b1394-d342-46c8-925c-595d66a359d4/graph/FlowchartEndpoint/3a657c8b-3068-401e-be60-79b6f99e547f):
```json
{ "MAPE": 0.1732282343612464 }
```

разница засчет округления ответа модели до 1000 и квантизации параметров в протобафе

### 3 service price -7%

In [80]:
preds = np.array(results)

In [81]:
y_true = np.exp(df_vals["target"].values.astype("float")) * 0.93

In [82]:
np.mean(np.abs(y_true - preds) / y_true)

0.12397075013890277

### 4 local model - 7%

In [51]:
preds = np.exp(cat.predict(df_vals[model_factors]))

In [49]:
y_true = np.exp(df_vals["target"].values.astype("float")) * 0.93

In [52]:
np.mean(np.abs(y_true - preds) / y_true)

0.12587567008841727

### 5 not rounded (only float -> int)

In [30]:
# @staticmethod
# def round_to_1000(val: float) -> int:
#     return int(val)

In [31]:
results = []
for i,row in df_vals.iterrows():
    offer_id = row["offer_id"]
    unified_address = unified_address_dict[int(offer_id)]
    prediction = get_service_prediction(row, offer_id, unified_address)
    results.append(prediction)

In [32]:
preds = np.array(results)

In [33]:
np.mean(np.abs(y_true - preds) / y_true)

0.17247710268846855

### 6  add 10% to not rounded (only float -> int)

In [34]:
preds = preds * 1.1

In [35]:
np.mean(np.abs(y_true - preds) / y_true)

0.1138863500674254

### 7 add 10% to service predictions

In [37]:
# @staticmethod
# def round_to_1000(val: float) -> int:
#     return max(1000, int((val * 1.1 + 500) / 1000) * 1000)

In [38]:
results = []
for i,row in df_vals.iterrows():
    offer_id = row["offer_id"]
    unified_address = unified_address_dict[int(offer_id)]
    prediction = get_service_prediction(row, offer_id, unified_address)
    results.append(prediction)

In [39]:
preds = np.array(results) # * 1.1 (on service side)

In [40]:
np.mean(np.abs(y_true - preds) / y_true)

0.11370035603837014

## check target drift

In [53]:
# https://nirvana.yandex-team.ru/flow/0dcedc37-d394-459a-8af1-2fd25e6b8fbb/35a665fe-7d39-40d8-94f3-39786189917f/graph/FlowchartEndpoint/73bc9ba4-d0ae-4d62-be14-cc9ea7b88da0

In [54]:
test_pool = pd.read_csv("test_pool.tsv", sep="\t")

In [55]:
y_test = np.exp(test_pool["RawFormulaVal"])
y_true = np.exp(test_pool["Label"])

In [56]:
np.mean(np.abs(y_true - y_test) / y_true)

0.14875877916027738

In [57]:
np.mean(y_true - y_test)

4993.715890251546

In [58]:
np.mean(y_true / y_test)

1.1086739417087692

try to use drift

In [59]:
np.mean(np.abs(y_true - y_test * 1.1) / y_true)

0.15411277262058506

In [60]:
np.mean(np.abs(y_true - (y_test + 5000)) / y_true)

0.2520855445643462

In [61]:
np.mean(np.abs(y_true - y_test * 1.05) / y_true)

0.145888365566656